# Problema de la Mochila
##### Autor: Pamela Bustamante

Descripción del problema: Tenemos $n$ artículos y una mochila de capacidad $V$. Cada artículo $i$ tiene una determinada utilidad $u_i$ y peso $v_i$. Queremos elegir qué artículos llevar en la mochila, maximizando la utilidad obtenida y sin sobrepasar la capacidad de la mochila.

Vamos a definir un modelo de optimización para solucionar este problema.

#### Definición de Variables

$$ x_i=\left\{
\begin{array}{ll}
      1 & \text{Si el artículo i se coloca en la mochila} \\
      0 & \text{En otro caso} \\
\end{array} \; \forall i \in [0,...,n]
\right.  $$

#### Modelo de optimización
\begin{align}
\text{max } & \sum_i u_i x_i \\
\text{s.a} & \\
&\sum_i v_i x_i \le V & \forall i \in [0,...,n]\\
& x_i \in {0,1} & \forall i \in [0,...,n] \\
\end{align}

#### Modelo en Python + Cplex

In [1]:
import random
import cplex  

Se define función para generar pesos y utilidades de artículos

In [2]:
def generate_instance(n):
    #Se definen utilidades y peso de artículos, y límite máximo de peso de mochila 

    u,v = {}, {}
    for i in range(n):
        u[i] = random.randint(0,100)
        v[i] = random.randint(0,100)

    C = random.uniform(0,1)*sum(v[i] for i in range(n))
    return u,v,C


Se define función para generar modelo

In [3]:
def mochila(n, V, u, v): 

    #Definición de modelo.
    model = cplex.Cplex() # Se crea un object model.
    model.set_problem_name("Knapsack")
    
    #B = binario
    #obj representa los coeficientes de las variables en la funcion objetivo
    #Valor obj por defecto= 0
    model.variables.add(names=["x[{0}]".format(i) for i in range(n)], types = ['B']*n, obj = [u[i] for i in range(n)])
    

    #Se declara que problema es de maximización. Por defecto, problema es de minimizacion
    model.objective.set_sense(model.objective.sense.maximize)

    #Se declara la restricción de capacidad
    coef = [v[i] for i in range(n)]  # coef e índices deben ser de la misma dimensión, en otro caso se genera un error.
    indices = ["x[{0}]".format(i) for i in range(n)]
    sumq = cplex.SparsePair(ind=indices,val=coef)  #Se declara una expresión lineal
    print(sumq)
    print(V)
    model.linear_constraints.add(lin_expr=[sumq],senses='L',rhs=[V]) # L menor o igual; G mayor o igual; E igual
    
    return model


Ejemplo de un problema de mochila

In [4]:
#######################
# Ejemplo
#######################
    
#Resolvemos el problema con n objetos
n = 10

#Se genera una instancia aleatoria con la función definida anteriormente
u,v,V = generate_instance(n)


#Se crea un modelo
mod = mochila(n, V,u,v)

#Se resuelve el modelo
mod.solve()

#Resultados
for i in range(n):
    print("x[{0}]={1}".format(i,mod.solution.get_values("x[{0}]".format(i))))
    
print('Fración de los objetos que llevamos',len([i for i in range(n) if mod.solution.get_values("x[{0}]".format(i)) == 1])/n)

#Valor objetivo
print('Valor objetivo', mod.solution.get_objective_value())

SparsePair(ind = ['x[0]', 'x[1]', 'x[2]', 'x[3]', 'x[4]', 'x[5]', 'x[6]', 'x[7]', 'x[8]', 'x[9]'], val = [34, 97, 27, 81, 19, 83, 11, 58, 60, 61])
159.04953160401115
Version identifier: 12.10.0.0 | 2019-11-27 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve added 1 rows and 1 columns.
MIP Presolve modified 1 coefficients.
Reduced MIP has 2 rows, 11 columns, and 13 nonzeros.
Reduced MIP has 10 binaries, 1 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.01 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 1 rows and 1 columns.
MIP Presolve added 1 rows and 1 columns.
Reduced MIP has 2 rows, 11 columns, and 13 nonzeros.
Reduced MIP has 10 binaries, 1 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.01 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Clique table members: 1.
MIP emphasis: balance optimal